# Анализ результатов экспериментов от 28.08.25 и 29.08.25

In [ ]:
%load_ext autoreload
%autoreload 2

## Описание эксперимента

Использовался алгоритм TD3 (Twin Delayed DDPG), адаптированный для работы в реальном времени в среде с бесконечным горизонтом. Состояние системы $s$ описывалось значениями с АЦП (управляемый сигнал) и ЦАП (управляющий сигнал), а также заданным для поддержания значением. Действие $a$ агента представляло собой выбор коэффициентов ПИД-регулятора $(K_p, K_i, K_d)$, а награда $r$ определялась по формуле
$$
r = -\left\lvert U_\text{setpoint} - U_\text{process variable} \right\rvert,
$$
где $U_\text{setpoint}$ — установленное значение, а $U_\text{process variable}$ - сигнал с АЦП.

Для корректной работы в реальном времени процесс сбора данных (взаимодействие агента с системой и формирование буфера переходов) и процесс обучения (обновления Q-функций и политики) были разделены.

Actor использовал LSTM + MLP, Critic только MLP.

## Результаты

In [ ]:
from pathlib import Path

EXPERIMENT_NAME = "td3_train_real_async"
EXPERIMENT_DATE = "2025-08-28"
EXPERIMENT_TIME = "15-38-42"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}/{EXPERIMENT_TIME}")

LOG_TRAIN_DIR = PATH_TO_EXP_DIR / "train_logs"
LOG_ENV_DIR = PATH_TO_EXP_DIR / "env_logs"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_tensorboard_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

In [ ]:
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator

def tensorboard_to_df(logdir : str):
    ea = event_accumulator.EventAccumulator(logdir)
    ea.Reload()

    tags = ea.Tags()["scalars"]

    all_data = []

    for tag in tags:
        events = ea.Scalars(tag)
        for e in events:
            all_data.append({
                "wall_time": e.wall_time,
                "step": e.step,
                "tag": tag,
                "value": e.value
            })

    df = pd.DataFrame(all_data)
    return df.pivot(index="step", columns="tag", values="value").reset_index()

train_df = tensorboard_to_df(str(LOG_TRAIN_DIR))

In [ ]:
env_df = tensorboard_to_df(str(LOG_ENV_DIR))
env_df["Reward"] = -abs(env_df["Observation/x"] - env_df["Observation/setpoint"]) + 1

In [ ]:
step_max = env_df["step"].max()
EXPERIMENT_TIME_MIN = 119
total_seconds = EXPERIMENT_TIME_MIN * 60  

env_df["time_sec"] = env_df["step"] / step_max * total_seconds

Tensorboard содержит не все данные: количество step значительно количества точек.

In [ ]:
print(env_df.info())
print(env_df["step"])

In [ ]:
print(train_df.info())
print(train_df["step"])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, train_df, tags=["Loss/Critic", "Loss/Actor"], title="Training curves")
plt.tight_layout()
plt.savefig(LOG_TRAIN_DIR / "train_logs.pdf")

In [ ]:
env_df["Observation/x"] = env_df["Observation/x"].apply(lambda x : ((x + 1.0) / 2) * 10230)
env_df["Observation/setpoint"] = env_df["Observation/setpoint"].apply(lambda x : ((x + 1.0) / 2) * 10230)

## График для ВКВО

In [ ]:
plt.rcParams.update({
    'font.size': 14,          # шрифт для подписей осей и текста
    'axes.labelsize': 16,     # шрифт для подписей осей
    'axes.titlesize': 18,     # шрифт заголовка графика
    'xtick.labelsize': 14,    # шрифт делений по X
    'ytick.labelsize': 14,    # шрифт делений по Y
    'legend.fontsize': 14     # шрифт легенды
})

SETPOINT = 1200

fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.plot(env_df["time_sec"], env_df["Observation/x"], color="green", label=r"$U_{\text{АЦП}}$")
ax1.axhline(SETPOINT, color="black", linestyle="--")
ax1.set_xlabel("Время (с)")
ax1.set_ylabel("Напряжение на фотодетекторе (АЦП)", color="green")
ax1.tick_params(axis='y', labelcolor="green")
ax1.set_ylim(0, 2000)
ax1.set_xlim(1, 4000)

# lines, labels = ax1.get_legend_handles_labels()
# ax1.legend(lines, labels, loc="upper right")

plt.tight_layout()

График для отчета по ВКР.

In [ ]:
plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14
})

SETPOINT = 1200

fig, ax1 = plt.subplots(figsize=(10, 6))

u_mean = env_df["Observation/x"].mean()
u_std = env_df["Observation/x"].std()
print(f"U mean = {u_mean}; U std = {u_std}")

ax1.plot(env_df["time_sec"], env_df["Observation/x"], color="green", label=r"$U_{\text{АЦП}}$")
ax1.axhline(SETPOINT, color="black", linestyle="--")
ax1.set_xlabel("Время (с)")
ax1.set_ylabel("Напряжение на фотодетекторе (АЦП)", color="green")
ax1.tick_params(axis='y', labelcolor="green")
ax1.set_ylim(0, 2000)
ax1.set_xlim(1, 4000)

ax1.grid(True, alpha=0.3)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint"], title="Observation curves")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "observation_logs.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["Reward"], title="Reward curve")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "reward_logs.pdf")

In [ ]:
action_tags = ["Action/kp", "Action/ki", "Action/kd"]

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, 10000)

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}_first_steps.pdf")

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}.pdf")

График для ВКВО

In [ ]:
from nn_laser_stabilizer.envs.constants import DEFAULT_KD

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(env_df["time_sec"], env_df["Action/kd"], color="blue", label=r"Предсказанное моделью значение $K_{\text{d}}$")
ax.axhline(DEFAULT_KD, color="black", linestyle="--", label=r"Текущее значение $K_{\text{d}}$")
ax.set_xlabel("Время (с)")
ax.set_ylabel(r"Коэффициент $K_{\text{d}}$", color="blue")
ax.tick_params(axis='y', labelcolor="blue")
ax.set_xlim(0, 300)

lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc="best")

plt.tight_layout()
plt.savefig("kd.png")